In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import win32com.client
import getpass




In [4]:
import win32com.client

# Initialize SAP GUI scripting engine
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine

# Get the first connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the SAP window
session.findById("wnd[0]").maximize()

# Enter transaction code ME2L and execute
session.findById("wnd[0]/tbar[0]/okcd").text = "me2l"
session.findById("wnd[0]").sendVKey(0)

# Set the "we101" text in the appropriate field
session.findById("wnd[0]/usr/ctxtSELPA-LOW").text = "we101"

# Set focus and position for the next field
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").caretPosition = 0

# Press the value push button
session.findById("wnd[0]/usr/btn%_S_BSART_%_APP_%-VALU_PUSH").press()

# Fill in multiple values in a subsequent popup
values = ["zsto", "zspt", "zvor", "zipl", "zatt"]
for i, value in enumerate(values):
    session.findById(f"wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,{i}]").text = value

# Set focus and caret position
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").caretPosition = 4

# Send VKey 8 to confirm the entries
session.findById("wnd[1]").sendVKey(8)

# Set the plant to "0201"
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").text = "0201"
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").caretPosition = 4

# Send VKey 8 to confirm the plant entry
session.findById("wnd[0]").sendVKey(8)

# Execute the next steps
session.findById("wnd[0]").sendVKey(23)
session.findById("wnd[0]/tbar[1]/btn[33]").press()

# Interact with the subsequent popup
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(8, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").firstVisibleRow = 3
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "8"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set file path and name
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)\\"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "me2l_r3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 7
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)




In [5]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/me2l_r3.XLSX")
    book.close()
except Exception as e:
    print(e)

LECTURA DE DATAFRAMES

In [6]:

#ME2L POR SEPARADO
me2l_df = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/me2l_r3.xlsx", dtype={'Posición':'str', 'Documento compras':'str'})




In [7]:
me2l_df = me2l_df.drop(0)

Estructura de ME2L

In [9]:
base = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros/Base Planificable/2024-08 Base Planificable/R3/Base Agosto OEM-AXS.xlsx", header=1)
base_df_prov = base[['Proveedor', 'Pais (Proveedor)']]
base_df = base[['Material','Texto breve de material','NomSector_actual','TIPO','Corresponde','Proveedor', 'Pais (Proveedor)']]

ESTRUCTURA BASE PLANIFICABLE

In [10]:
from pathlib import Path
#BASE PLANIFICABLE
hoy = datetime.datetime.today()
usuario = getpass.getuser()
ruta = f'C:/Users/{usuario}/Inchcape/Planificación y Compras Chile - Documentos/Bases Indicadores en CSV {hoy.year}-{hoy.month:02d}'
ruta_repo = Path(ruta)
base_df = base_df[['Material','Texto breve de material','NomSector_actual','TIPO','Corresponde']]

In [11]:
columnas= ['Nro_pieza_fabricante_1',	'Cod_Actual_1']

#columnas= ['Nro_pieza_fabricante_1',	'Cod_Actual_1']
ruta_cod = ruta_repo.joinpath('COD_ACTUAL.csv')

# Leer el archivo CSV en un DataFrame
cod_actual_df = pd.read_csv(ruta_cod)
cod_actual_df = cod_actual_df[columnas]

In [12]:
base_df_ue = pd.merge(base_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")

In [13]:
base_df_ue['Cod_Actual_1'] = base_df_ue['Cod_Actual_1'].fillna(base_df_ue['Material'])

In [14]:
base_df_ue = base_df_ue[['Cod_Actual_1','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']]
base_df_ue = base_df_ue.rename(columns={'Cod_Actual_1':'Material'})

In [15]:
base_df_ue = base_df_ue.drop_duplicates(subset=['Material'])

TRATAMIENTO DE DFS (OC, POSiCION)

In [16]:
print(f'Dimensiones del df1: \nFILAS:{me2l_df.shape[0]}\nCOLUMNAS: {me2l_df.shape[1]}')



Dimensiones del df1: 
FILAS:18408
COLUMNAS: 37


In [17]:
me2l_df['Material'] = me2l_df['Material'].astype('str')
me2l_df['Material'] = me2l_df['Material'].apply(lambda x: x.split(".")[0])

ULTIMO ESLABON A ME2L

In [18]:
me2l_ue = pd.merge(me2l_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")

In [19]:
me2l_ue['Cod_Actual_1'].fillna(me2l_ue['Material'], inplace=True)

In [20]:
me2l_ue = me2l_ue[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material','Cod_Actual_1', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',  'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)'
       ]]


In [21]:
me2l_ue = me2l_ue.rename(columns={'Material':'Material Antiguo'})
me2l_ue = me2l_ue.rename(columns={'Cod_Actual_1':'Material'})


In [22]:
me2l_df['Material'] = me2l_df['Material'].astype('str')
me2l_df['Material'] = me2l_df['Material'].apply(lambda x: x.split(".")[0])

me2l_ue = pd.merge(me2l_df, cod_actual_df, left_on="Material", right_on="Nro_pieza_fabricante_1", how="left")
me2l_ue['Cod_Actual_1'].fillna(me2l_ue['Material'], inplace=True)

me2l_ue = me2l_ue[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material','Cod_Actual_1', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Cantidad base',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',  'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)'
       ]]

me2l_ue = me2l_ue.rename(columns={'Material':'Material Antiguo'})
me2l_ue = me2l_ue.rename(columns={'Cod_Actual_1':'Material'})

base_df_prov = base_df_prov.drop_duplicates(subset=['Proveedor'])
me2l_cruce_sector = pd.merge(me2l_ue, base_df_ue[['Material','Texto breve de material', 'NomSector_actual', 'TIPO','Corresponde']], left_on="Material", right_on="Material", how="left")
#me2l_ue.to_excel("C:/Users/lravlic/PROYECTOS DATA/PRUEBAS TRANSITO/me2l_sector.xlsx")
me2l_cruce_sector.shape
me2l_cruce_sector = me2l_cruce_sector[['Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 
       'Fecha de entrega','Hora', 'Fecha entrega estad.',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual',
       'Texto breve de material', 'TIPO', 'Corresponde']]
me2l_cruce_sector['Cod_Prov'] = me2l_cruce_sector['Nombre del proveedor'].str.split(' ', expand=True)[0]
me2l_cruce_sector = me2l_cruce_sector.merge(base_df_prov,left_on='Cod_Prov', right_on='Proveedor', how='left')
me2l_cruce_sector['Posición'] = me2l_cruce_sector['Posición'].astype('str')
me2l_cruce_sector['AUX'] = me2l_cruce_sector['Documento compras'] + me2l_cruce_sector['Posición']
me2l_cruce_sector['Origen'] = me2l_cruce_sector['Pais (Proveedor)']
me2l_cruce_sector = me2l_cruce_sector[['AUX','Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda',
       'Fecha de entrega','Hora', 'Fecha entrega estad.',
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual','Origen', 'TIPO',  'Corresponde']]
me2l_base_tr = me2l_cruce_sector[me2l_cruce_sector['NomSector_actual'].notna()]
me2l_base_tr['Corresponde'].fillna(0, inplace=True)
me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)
me2l_base_tr

C:\Users\lravlic\AppData\Local\Temp\ipykernel_27184\889298065.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].fillna(0, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_27184\889298065.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)


,AUX,Documento compras,Posición,Reparto,Cl.documento compras,Tipo doc.compras,Grupo de compras,Historial pedido/Docu.orden entrega,Fecha documento,Material Antiguo,...,Cantidad de posiciones,Nombre del proveedor,Por entregar (cantidad),Por entregar (valor),Por calcular (cantidad),Por calcular (valor),NomSector_actual,Origen,TIPO,Corresponde
0,430001539910,4300015399,10,1.0,ZVOR,F,RR4,NaN,2024-05-15,S203F2602060605,...,1,10000717 SAMUEL SALVADOR SUAREZ C,4.0,638320.00,4.0,638320.00,Changan,CL,OEM,1.0
1,430001625610,4300016256,10,1.0,ZVOR,F,RR4,NaN,2024-06-24,4311017560R15CPZ,...,1,10000717 SAMUEL SALVADOR SUAREZ C,4.0,206720.00,4.0,206720.00,Suzuki,CL,OEM,1.0
2,420000568910,4200005689,10,1.0,ZSTO,F,RR2,NaN,2024-06-11,PORTADOCSUZUKI,...,1,10001447 I.V. Fabrica de producto,400.0,1192000.00,1135.0,3382300.00,Accesorios,CL,AXS,1.0
3,420000591710,4200005917,10,1.0,ZSTO,F,RR2,NaN,2024-07-10,PORTADCTOKYAL,...,1,10001447 I.V. Fabrica de producto,852.0,3493200.00,852.0,3493200.00,Accesorios,CL,AXS,1.0
4,420000591740,4200005917,40,1.0,ZSTO,F,RR2,NaN,2024-07-10,PORTADOCPOER,...,1,10001447 I.V. Fabrica de producto,225.0,905625.00,225.0,905625.00,Accesorios,CL,AXS,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18403,4300017115360,4300017115,360,1.0,ZVOR,F,RR9,NaN,2024-08-05,8210100U1910-F011,...,1,PU02 DERCO PERU S.A.,1.0,12.81,1.0,12.81,JAC Cars,NaN,OEM,1.0
18404,4300017115370,4300017115,370,1.0,ZVOR,F,RR9,NaN,2024-08-05,Z-1701114-00-00,...,1,PU02 DERCO PERU S.A.,1.0,3.23,1.0,3.23,JAC Cars,NaN,OEM,1.0
18405,4300017115380,4300017115,380,1.0,ZVOR,F,RR9,NaN,2024-08-05,ZM6T82AF-1701513,...,1,PU02 DERCO PERU S.A.,1.0,3.56,1.0,3.56,JAC Cars,NaN,OEM,1.0
18406,4300017115390,4300017115,390,1.0,ZVOR,F,RR9,NaN,2024-08-05,6203400XSZ08A,...,1,PU02 DERCO PERU S.A.,1.0,30.16,1.0,30.16,Great Wall,NaN,OEM,1.0


In [23]:
carpeta_tubo = "C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Tubo Semanal"
ruta_tubo = os.listdir(carpeta_tubo)[-2]

destino_tubo = carpeta_tubo + '/' +ruta_tubo + '/' + ruta_tubo  +' - '

me2l_base_tr.to_excel(destino_tubo + 'ME2L R3.xlsx')

In [24]:
me2l_cruce_sector['AUX'] = me2l_cruce_sector['Documento compras'] + me2l_cruce_sector['Posición']

In [25]:
me2l_cruce_sector = me2l_cruce_sector[['AUX','Documento compras', 'Posición', 'Reparto','Cl.documento compras','Tipo doc.compras',
       'Grupo de compras','Historial pedido/Docu.orden entrega', 'Fecha documento', 'Material Antiguo','Material', 'Texto breve','Grupo de artículos',
       'Indicador de borrado','Tipo de posición', 'Tipo de imputación', 'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda',
       'Fecha de entrega','Hora', 'Fecha entrega estad.', 
       'Cantidad entrada', 'Cantidad de salida', 'Cantidad entregada',
       'Solicitud de pedido', 'Pos.solicitud pedido', 'Indicador creación',
       'Cantidad de posiciones', 'Nombre del proveedor',
       'Por entregar (cantidad)', 'Por entregar (valor)',
       'Por calcular (cantidad)', 'Por calcular (valor)', 'NomSector_actual','Origen', 'TIPO',  'Corresponde']]

In [26]:
me2l_base_tr = me2l_cruce_sector[me2l_cruce_sector['NomSector_actual'].notna()]

In [27]:
me2l_base_tr['Corresponde'].fillna(0, inplace=True)
me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_27184\3910453820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].fillna(0, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_27184\3910453820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me2l_base_tr['Corresponde'].replace({0:1}, inplace=True)


In [28]:
bases_oc = me2l_base_tr.groupby(['Documento compras'])['Posición'].count().sort_values(ascending=False).reset_index()

In [29]:


# Calculate the number of rows in each part
num_rows_per_part = len(bases_oc) // 2  # Integer division to get an equal split

# Split the DataFrame into five parts
df_parts = np.array_split(bases_oc, 2)


# Alternatively, you can access the parts using indexing (e.g., df_parts[0], df_parts[1], etc.)

# Each part will have a similar number of rows


c:\Users\lravlic\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [30]:
df_parts[0]['Documento compras'].to_clipboard(header=False, index=False)

In [31]:
import win32com.client

# Initialize SAP GUI scripting engine
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine

# Get the first connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the SAP window
session.findById("wnd[0]").maximize()

# Enter transaction code ZMM_MONITOR_ORDEN_CL and execute
session.findById("wnd[0]/tbar[0]/okcd").text = "ZMM_MONITOR_ORDEN_CL"
session.findById("wnd[0]").sendVKey(0)

# Select the specific tab
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC").select()

# Set focus and position for the required field
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").setFocus()
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").caretPosition = 0

# Press the value push button
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/btn%_SO_OC_%_APP_%-VALU_PUSH").press()

# Interact with the buttons in the new window
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()

# Send VKey 8 to confirm the entries
session.findById("wnd[0]").sendVKey(8)

# Press the toolbar button
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarButton("&MB_VARIANT")

# Interact with the subsequent popup
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(3, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "3"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()

# Export the data
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").selectContextMenuItem("&XXL")
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set the file path and name
session.findById("wnd[1]/usr/ctxtDY_PATH").text = ""
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "monitor_1_r3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]").sendVKey(4)

# Set the directory path in the new window
session.findById("wnd[2]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)"
session.findById("wnd[2]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[2]/usr/ctxtDY_PATH").caretPosition = 131
session.findById("wnd[2]/tbar[0]/btn[11]").press()

# Confirm the export
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)


In [32]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/monitor_1_r3.XLSX")
    book.close()
except Exception as e:
    print(e)

In [33]:
df_parts[1]['Documento compras'].to_clipboard(header=False, index=False)

In [34]:
import win32com.client

# Initialize SAP GUI scripting engine
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine

# Get the first connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the SAP window
session.findById("wnd[0]").maximize()

# Enter transaction code ZMM_MONITOR_ORDEN_CL and execute
session.findById("wnd[0]/tbar[0]/okcd").text = "ZMM_MONITOR_ORDEN_CL"
session.findById("wnd[0]").sendVKey(0)

# Select the specific tab
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC").select()

# Set focus and position for the required field
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").setFocus()
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/ctxtSO_OC-LOW").caretPosition = 0

# Press the value push button
session.findById("wnd[0]/usr/tabsTABSTRIP_TABB2/tabpMON_OC_OC/ssub%_SUBSCREEN_TABB2:ZMM_MONITO3_SEGUIMIENTO_ORV4CL:1003/btn%_SO_OC_%_APP_%-VALU_PUSH").press()

# Interact with the buttons in the new window
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()

# Send VKey 8 to confirm the entries
session.findById("wnd[0]").sendVKey(8)

# Press the toolbar button
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarButton("&MB_VARIANT")

# Interact with the subsequent popup
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(3, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "3"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()

# Export the data
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
session.findById("wnd[0]/usr/cntlCONTROL_ALV_UPPER/shellcont/shell").selectContextMenuItem("&XXL")
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set the file path and name
session.findById("wnd[1]/usr/ctxtDY_PATH").text = ""
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "monitor_2_r3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]").sendVKey(4)

# Set the directory path in the new window
session.findById("wnd[2]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)"
session.findById("wnd[2]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[2]/usr/ctxtDY_PATH").caretPosition = 131
session.findById("wnd[2]/tbar[0]/btn[11]").press()

# Confirm the export
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)


In [35]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/monitor_2_r3.XLSX")
    book.close()
except Exception as e:
    print(e)

In [36]:
bases_oc['Documento compras'].to_clipboard(header=False, index=False)

In [37]:
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine
connection = application.Children(0)
session = connection.Children(0)
#now = datetime.now()

#session.findById("wnd[0]").maximize
#session.findById("wnd[0]").maximize
session.findById("wnd[0]/tbar[0]/okcd").text = "vl06if"
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").text = ""
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").text = ""
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtIT_LFDAT-LOW").caretPosition = 0
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").setFocus()
session.findById("wnd[0]/usr/ctxtIT_LFDAT-HIGH").caretPosition = 0
session.findById("wnd[0]").sendVKey(0)
session.findById("wnd[0]/usr/btn%_IT_EBELN_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()
session.findById("wnd[0]").sendVKey(8)
session.findById("wnd[0]/tbar[1]/btn[18]").press()
session.findById("wnd[0]").sendVKey(33)
session.findById("wnd[1]/usr/lbl[1,7]").setFocus()
session.findById("wnd[1]/usr/lbl[1,7]").caretPosition = 4
session.findById("wnd[1]").sendVKey(2)
session.findById("wnd[0]").sendVKey(43)
session.findById("wnd[1]/usr/cmbG_LISTBOX").setFocus()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Inchcape\\Planificación y Compras Chile - Documentos\\Planificación y Compras KPI-Reportes\\Bases Transito (python)"
session.findById("wnd[1]/tbar[0]/btn[11]").press()
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)




In [38]:
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/export.XLSX")
    book.close()
except Exception as e:
    print(e)

In [39]:
lista = os.listdir("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)")

In [40]:
dfs = []
print('Archivos usados (Monitor)')
for i in lista:
    if ('monitor_' in i):
        df = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)" + '/' +i, index_col=None)
        print(i)
        dfs.append(df)


Archivos usados (Monitor)
monitor_1_r3.XLSX
monitor_2_r3.XLSX


In [41]:
df_ee = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/export.XLSX" ,dtype={'Documento compras':'str','Posición modelo':'str'})

In [42]:
print(f'Dimensiones del df1\n: FILAS:{df_ee.shape[0]}\nCOLUMNAS: {df_ee.shape[1]}')

Dimensiones del df1
: FILAS:41675
COLUMNAS: 13


In [43]:
monitor_df = pd.concat(dfs,axis=0, ignore_index=True)

In [44]:
monitor_df

,Prefijo OC,Nro. OC SAP,Clase Doc. OC,Posición OC SAP,Ind. Borrado OC,Fecha Creación OC,Clase Doc. SP,Nro. SOLPED,Posición SOLPED,Ind. Borrado SP,...,Días del pedido,Días en BO,Días Pend. Emb.,Pedido Cerrado,Material Cotizado,Cód. Mat de prov en PO,Cantidad en BO,Pet.Ofert.Creado por,OC Creado por,"Marcar para ""Cerrar"""
0,SIM003-24,4200004288,ZSTO,3930,NaN,2024-01-12,ZSTO,13608391.0,7840,NaN,...,206,0,0,Sí,82801M69R50WAA,82801M69R50-WAA,0,JFIGUEROAP,JFIGUEROAP,Indeterminado
1,SIM003-24,4200004288,ZSTO,4020,NaN,2024-01-12,ZSTO,13608391.0,7930,NaN,...,206,0,0,Sí,82802M69R50WAA,82802M69R50-WAA,0,JFIGUEROAP,JFIGUEROAP,Indeterminado
2,SIM003-24,4200004288,ZSTO,4060,NaN,2024-01-12,ZSTO,13608391.0,7970,NaN,...,206,0,0,Sí,82802M69R50ZTN,82802M69R50-ZTN,0,JFIGUEROAP,JFIGUEROAP,Indeterminado
3,SIM003-24,4200004288,ZSTO,4070,NaN,2024-01-12,ZSTO,13608391.0,7990,NaN,...,206,0,0,Sí,82820M68P20ZTN,82820M68P20-ZTN,0,JFIGUEROAP,JFIGUEROAP,Indeterminado
4,SIM003-24,4200004288,ZSTO,4080,NaN,2024-01-12,ZSTO,13608391.0,8000,NaN,...,206,0,0,Sí,82820M72R20Z2S,82820M72R20-Z2S,0,JFIGUEROAP,JFIGUEROAP,Indeterminado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54141,JDM003-23,4200004531,ZSTO,10,NaN,2024-01-25,NaN,NaN,0,NaN,...,0,0,0,No,NaN,NaN,21,NaN,E-LMEZA,Indeterminado
54142,RFM032-23,4200004378,ZSTO,60,NaN,2024-01-16,NaN,NaN,0,NaN,...,0,0,0,No,NaN,NaN,1,NaN,E-LMEZA,Indeterminado
54143,CHLJACM028-24,4200005755,ZSTO,10,NaN,2024-06-22,ZSTO,13621900.0,830,NaN,...,48,18,0,No,7911140U3400,7911140U3400,6,APAINEO,APAINEO,Indeterminado
54144,CHLJACM028-24,4200005755,ZSTO,20,NaN,2024-06-22,ZSTO,13621900.0,860,NaN,...,48,18,0,No,7911141P3010,7911141P3010,6,APAINEO,APAINEO,Indeterminado


In [45]:
monitor_df.to_excel(destino_tubo + "Monitor R3.xlsx")


In [46]:
df_ee.to_excel(destino_tubo + "Entrega Entrante R3.xlsx")

In [47]:
#monitor_df = monitor_df[['Nro. OC SAP','Clase Doc. OC','Posición OC SAP','Ind. Borrado OC','Clase Doc. SP','Nro. SOLPED','Posición SOLPED','Ind. Borrado SP','Cant. UN. OC','N° MIRO','Posición Factura',
#'Cant. Factura','Fe. Confirmación','Fe. Compromiso','Fe. ATA','TF de ATA','Cantidad en BO','Ind. Borrado PO']]

In [48]:
monitor_df['Nro. OC SAP'] = monitor_df['Nro. OC SAP'].astype('str')
monitor_df['Posición OC SAP'] = monitor_df['Posición OC SAP'].astype('str')


In [49]:
monitor_df['AUX'] = monitor_df['Nro. OC SAP'] + monitor_df['Posición OC SAP']

In [50]:
monitor_df['N° MIRO'] = monitor_df['N° MIRO'].astype('str')

In [51]:
#revision = monitor_df.groupby('AUX')['Fe. ATA'].nunique().sort_values(ascending=False).reset_index()

In [52]:
tabla_1 = monitor_df[monitor_df['Ind. Borrado OC'].isna() &
    monitor_df['Ind. Borrado SP'].isna() &
    monitor_df['Ind. Borrado PO'].isna()].groupby(['AUX', 'N° MIRO']).agg({'Cant. UN. OC':'mean', 'Cant. Factura':'max','Cantidad en BO':'mean'})

In [53]:
tabla_final = tabla_1.groupby(['AUX']).agg({'Cant. UN. OC':'mean','Cant. Factura':'sum', 'Cantidad en BO':'mean'}).reset_index()

In [54]:
print(f'Dimension de tabla final: {tabla_final.shape}')
print('-'*40)
print(f'Total Campo OC: {tabla_final['Cant. UN. OC'].sum()}')
print(f'Total Campo Factura: {tabla_final['Cant. Factura'].sum()}')
print(f'Total Campo BO: {tabla_final['Cantidad en BO'].sum()}')

Dimension de tabla final: (45050, 4)
----------------------------------------
Total Campo OC: 613576.0
Total Campo Factura: 451806
Total Campo BO: 159291.0


In [55]:
tabla_final[tabla_final['AUX']=='420000494760']

,AUX,Cant. UN. OC,Cant. Factura,Cantidad en BO


In [56]:
monitor_df.shape

(54146, 107)

In [57]:
df_ee.columns

Index(['Entrega', 'Posición', 'Material', 'Denominación', 'Cantidad entrega',
       'Un.medida venta', 'Fe.entr.mercancías', 'Posición modelo', 'Proveedor',
       'Documento modelo', 'Nombre del proveedor', 'Documento compras',
       'Status mov.mcía.'],
      dtype='object')

In [58]:
df_ee = df_ee[['Documento compras','Posición modelo', 'Cantidad entrega', 'Status mov.mcía.']]

In [59]:
df_ee['Documento compras'] = df_ee['Documento compras'].astype('str')
df_ee['Posición modelo'] = df_ee['Posición modelo'].astype('str')

In [60]:
df_ee.dtypes

Documento compras    object
Posición modelo      object
Cantidad entrega      int64
Status mov.mcía.     object
dtype: object

In [61]:
df_ee['AUX'] = df_ee['Documento compras'] + df_ee['Posición modelo']
df_ee.drop(columns=['Documento compras','Posición modelo'])

,Cantidad entrega,Status mov.mcía.,AUX
0,2,C,420000413360
1,1,C,420000413370
2,1,C,4200004133130
3,2,C,4200004133190
4,1,C,4200004133260
...,...,...,...
41670,1,A,430001712460
41671,1,A,430001712470
41672,1,A,430001712480
41673,32,NaN,nannan


In [62]:
table_ee = df_ee[df_ee['Status mov.mcía.']=='C'].groupby(['AUX'])['Cantidad entrega'].sum().reset_index()

In [63]:
me2l_base_tr.shape

(18340, 42)

In [64]:
me2l_15 = me2l_base_tr

In [65]:
monitor_df.columns

Index(['Prefijo OC', 'Nro. OC SAP', 'Clase Doc. OC', 'Posición OC SAP',
       'Ind. Borrado OC', 'Fecha Creación OC', 'Clase Doc. SP', 'Nro. SOLPED',
       'Posición SOLPED', 'Ind. Borrado SP',
       ...
       'Días en BO', 'Días Pend. Emb.', 'Pedido Cerrado', 'Material Cotizado',
       'Cód. Mat de prov en PO', 'Cantidad en BO', 'Pet.Ofert.Creado por',
       'OC Creado por', 'Marcar para "Cerrar"', 'AUX'],
      dtype='object', length=107)

In [66]:
monitor_df.drop_duplicates(subset=['AUX'], inplace=True)

In [67]:
me2l_15 = me2l_base_tr

In [68]:
me2l_15.dtypes

AUX                                            object
Documento compras                              object
Posición                                       object
Reparto                                       float64
Cl.documento compras                           object
Tipo doc.compras                               object
Grupo de compras                               object
Historial pedido/Docu.orden entrega           float64
Fecha documento                        datetime64[ns]
Material Antiguo                               object
Material                                       object
Texto breve                                    object
Grupo de artículos                            float64
Indicador de borrado                          float64
Tipo de posición                              float64
Tipo de imputación                            float64
Centro                                        float64
Almacén                                       float64
Cantidad de reparto         

In [69]:
tabla_final.reset_index()

,index,AUX,Cant. UN. OC,Cant. Factura,Cantidad en BO
0,0,4200004121100,1.0,1,0.0
1,1,4200004121110,1.0,1,0.0
2,2,4200004121120,1.0,1,0.0
3,3,4200004121130,1.0,1,0.0
4,4,4200004121140,1.0,1,0.0
...,...,...,...,...,...
45045,45045,470001887680,1.0,0,1.0
45046,45046,470001887690,1.0,0,1.0
45047,45047,470001891610,40.0,0,40.0
45048,45048,470001891620,40.0,0,40.0


In [70]:
me2l_15 = me2l_15.merge(monitor_df[['AUX','Fe. ATA']], left_on='AUX', right_on='AUX', how = 'left')

In [71]:
me2l_15['Fecha Est. Fact'] = np.where(me2l_15['Fe. ATA'].notnull(), me2l_15['Fe. ATA'] + pd.Timedelta(days=15), me2l_15['Fecha de entrega'])

In [72]:
tabla_final.head()

,AUX,Cant. UN. OC,Cant. Factura,Cantidad en BO
0,4200004121100,1.0,1,0.0
1,4200004121110,1.0,1,0.0
2,4200004121120,1.0,1,0.0
3,4200004121130,1.0,1,0.0
4,4200004121140,1.0,1,0.0


In [73]:
monitor_df.shape

(45376, 107)

In [74]:
me2l_15 = me2l_15.merge(tabla_final[['AUX', 'Cant. Factura']], left_on = 'AUX', right_on='AUX',how='left')

In [75]:
me2l_15['Cant. Factura'] = me2l_15['Cant. Factura'] * me2l_15['Corresponde']

In [76]:
me2l_15['Cant. Factura'].sum()

118321.0

In [77]:
table_ee.columns

Index(['AUX', 'Cantidad entrega'], dtype='object')

In [78]:
me2l_15= me2l_15.merge(table_ee, left_on='AUX', right_on='AUX', how='left')

In [79]:
table_ee.head()

,AUX,Cantidad entrega
0,4200004121100,1
1,4200004121110,1
2,4200004121120,1
3,4200004121130,1
4,4200004121140,1


In [80]:
me2l_15['Cantidad entrega'] = me2l_15['Cantidad entrega']*me2l_15['Corresponde']

In [81]:
me2l_15['Cantidad entrega'].sum()

14898.0

In [82]:
me2l_15['Qty por Entregar'] = me2l_15['Por entregar (cantidad)']*me2l_15['Corresponde']

In [83]:
me2l_15['Qty por Entregar'].sum()

243585.0

In [84]:
#cambiar esto por la fecha del dia actual

hoy = datetime.datetime.today()

In [85]:
me2l_15['Cantidad entrega'].fillna(0, inplace=True)

In [86]:


# Suponiendo que 'me2l_15' es tu DataFrame y 'AW4', 'BJ4' y 'AT4' son columnas en tu DataFrame
result = me2l_15['Cant. Factura'] - me2l_15['Cantidad entrega']
result = np.where(np.isnan(result), me2l_15['Qty por Entregar'], result)

me2l_15['Qty Fact Corr'] = np.where(result<0, 0, result)

In [87]:
me2l_15[['AUX','Qty Fact Corr']].to_excel('verif_x_entr.xlsx')

In [88]:
hoy = pd.to_datetime(hoy)

In [89]:
me2l_15[ 'Fecha Est. Fact'].max()

Timestamp('2024-12-11 00:00:00')

In [90]:
conditions = [
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] >= hoy),
    (me2l_15['Qty Fact Corr'] > 0) & (me2l_15['Fecha Est. Fact'] < hoy)
]

choices = ['Facturado No Vencido', 'Facturado Vencido']

me2l_15['Status TR Fact'] = np.select(conditions, choices, default='No Facturado')

In [91]:
me2l_15['Status TR Fact'].value_counts()

Status TR Fact
No Facturado            10822
Facturado No Vencido     7253
Facturado Vencido         265
Name: count, dtype: int64

In [92]:
from datetime import timedelta, datetime

hoy_31 = hoy  + timedelta(days=31)

In [93]:
hoy_31 = pd.to_datetime(hoy_31)

In [94]:
condition = (me2l_15['Fecha de entrega'] > hoy) & (me2l_15['Fecha de entrega'] < hoy_31)

# Apply the condition using boolean indexing and assign 'SI' or 'NO' accordingly
me2l_15['Fecha BO Mes Actual'] = np.where(condition, 'SI', 'NO')

In [95]:
me2l_15['Fecha BO Mes Actual'].value_counts()

Fecha BO Mes Actual
NO    14015
SI     4325
Name: count, dtype: int64

In [96]:
condition= (me2l_15['Fecha de entrega']< hoy)

me2l_15['TR Vencido'] = np.where(condition, 'SI','NO')

In [97]:
me2l_15['TR Vencido'].value_counts()

TR Vencido
NO    14543
SI     3797
Name: count, dtype: int64

In [98]:
conditions = [
    (me2l_15['Fecha BO Mes Actual'] == "NO") & (me2l_15['TR Vencido'] == "NO"),
    (me2l_15['TR Vencido'] == "SI"),
]

# Define choices
choices = [
    "Fecha Teórica",
    "TR Vencido",
]

# Apply np.select
me2l_15['Statur TR No FAct'] = np.select(conditions, choices, default="Replanificar +45")

In [99]:
me2l_15['Cant. Factura'].sum()

118321.0

In [100]:
#Qty BO
me2l_15['Cant. BO'] = me2l_15['Cant. Factura'].fillna(0, inplace=True)

In [101]:
#Cant Factura : Cant Fact CORR
#QTY BO: CHECK

In [102]:
condicion = (me2l_15['Fecha Est. Fact'] < hoy)

me2l_15['Atraso'] = np.where(condicion, 'SI','NO')

In [103]:
me2l_15['Atraso'].value_counts()

Atraso
NO    14816
SI     3524
Name: count, dtype: int64

In [104]:
me2l_15['Fecha Est. Fact']

0       2024-06-15
1       2024-06-24
2       2024-07-06
3       2024-08-04
4       2024-08-04
           ...    
18335   2024-09-19
18336   2024-09-19
18337   2024-09-19
18338   2024-09-19
18339   2024-09-19
Name: Fecha Est. Fact, Length: 18340, dtype: datetime64[ns]

In [105]:
condicion = (me2l_15['Atraso'] == 'SI')

me2l_15['Fecha Facturacion Final'] = np.where(condicion, hoy_31, me2l_15['Fecha Est. Fact'])



In [106]:
me2l_15['Fecha Facturacion Final']

0        2024-09-05 15:19:20.938111
1        2024-09-05 15:19:20.938111
2        2024-09-05 15:19:20.938111
3        2024-09-05 15:19:20.938111
4        2024-09-05 15:19:20.938111
                    ...            
18335           1726704000000000000
18336           1726704000000000000
18337           1726704000000000000
18338           1726704000000000000
18339           1726704000000000000
Name: Fecha Facturacion Final, Length: 18340, dtype: object

In [107]:
me2l_15['Fecha Facturacion Final'] = pd.to_datetime(me2l_15['Fecha Facturacion Final'])

In [108]:
me2l_15['Fecha Facturacion Final']

0       2024-09-05 15:19:20.938111
1       2024-09-05 15:19:20.938111
2       2024-09-05 15:19:20.938111
3       2024-09-05 15:19:20.938111
4       2024-09-05 15:19:20.938111
                   ...            
18335   2024-09-19 00:00:00.000000
18336   2024-09-19 00:00:00.000000
18337   2024-09-19 00:00:00.000000
18338   2024-09-19 00:00:00.000000
18339   2024-09-19 00:00:00.000000
Name: Fecha Facturacion Final, Length: 18340, dtype: datetime64[ns]

In [109]:
monitor_df.dtypes

Prefijo OC              object
Nro. OC SAP             object
Clase Doc. OC           object
Posición OC SAP         object
Ind. Borrado OC         object
                         ...  
Cantidad en BO           int64
Pet.Ofert.Creado por    object
OC Creado por           object
Marcar para "Cerrar"    object
AUX                     object
Length: 107, dtype: object

In [110]:
me2l_15 = me2l_15.merge(monitor_df[['AUX', 'Vía (Texto)']], left_on='AUX', right_on='AUX', how='left')

In [111]:
me2l_15['Vía (Texto)'].value_counts()

Vía (Texto)
Marítimo     12820
Aéreo         1706
Courier       1405
Terrestre      176
Name: count, dtype: int64

In [112]:
me2l_15.columns

Index(['AUX', 'Documento compras', 'Posición', 'Reparto',
       'Cl.documento compras', 'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Fecha de entrega',
       'Hora', 'Fecha entrega estad.', 'Cantidad entrada',
       'Cantidad de salida', 'Cantidad entregada', 'Solicitud de pedido',
       'Pos.solicitud pedido', 'Indicador creación', 'Cantidad de posiciones',
       'Nombre del proveedor', 'Por entregar (cantidad)',
       'Por entregar (valor)', 'Por calcular (cantidad)',
       'Por calcular (valor)', 'NomSector_actual', 'Origen', 'TIPO',
       'Corresponde', 'Fe. ATA', 'Fecha Est. Fact', 'Cant. Factura',
       'Cantidad entrega', 'Qty por Entreg

In [113]:
me2l_15['LT Objetivo'] = np.where(me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"]), 45,
                    np.where(me2l_15['Vía (Texto)'] == "Aéreo", 15,
                        np.where(me2l_15['Vía (Texto)'] == "Courier", 7, 0)))

In [114]:
me2l_15 = me2l_15.merge(monitor_df[['AUX','Fe. Confirmación']], left_on='AUX',right_on='AUX', how='left')

In [115]:
me2l_15['Fecha OC'] = np.where((me2l_15['Fe. Confirmación'] == 0) | (me2l_15['Fe. Confirmación'].isna()), me2l_15['Fecha documento'], me2l_15['Fe. Confirmación'])

In [116]:
import datetime
today = hoy

In [117]:
today = pd.to_datetime(today)

In [118]:


# Calculate the difference
me2l_15['Dias LT BO'] = (today - me2l_15['Fecha OC']).dt.days - me2l_15['LT Objetivo']

# Handling potential errors, setting them to 0
me2l_15['Dias LT BO'] = me2l_15['Dias LT BO'].fillna(0)

In [119]:
me2l_15['Dias LT BO'].sum()

543358

In [120]:
import pandas as pd
import numpy as np

# Define conditions and choices
conditions = [
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(1, 30)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(31, 60)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'].between(61, 90)),
    (me2l_15['Vía (Texto)'].isin(["Marítimo", "Terrestre"])) & (me2l_15['Dias LT BO'] > 90),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(1, 15)),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(16, 30)),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'].between(31, 45)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(1, 7)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(8, 15)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'].between(16, 21)),
    (me2l_15['Vía (Texto)'] == "Courier") & (me2l_15['Dias LT BO'] > 21),
    (me2l_15['Vía (Texto)'] == "Aéreo") & (me2l_15['Dias LT BO'] > 45)
]

choices = [
    "De 1 a 30 días",
    "De 31 a 60 días",
    "De 61 a 90 días",
    "De 91 días a más",
    "De 1 a 15 días",
    "De 16 a 30 días",
    "De 31 a 45 días",
    "De 1 a 7 días",
    "De 8 a 15 días",
    "De 16 a 21 días",
    "De 21 días a más",
    "De 45 días a más"
]

# Assign the values based on conditions
me2l_15['Categoría LT'] = np.select(conditions, choices, default="Dentro de LT")



In [121]:
me2l_15['Categoría LT'].value_counts()

Categoría LT
Dentro de LT        7079
De 1 a 30 días      4030
De 31 a 60 días     2526
De 61 a 90 días     1666
De 91 días a más     644
De 45 días a más     569
De 1 a 15 días       316
De 1 a 7 días        314
De 21 días a más     299
De 16 a 30 días      288
De 31 a 45 días      261
De 8 a 15 días       189
De 16 a 21 días      159
Name: count, dtype: int64

In [122]:
import pandas as pd
import numpy as np

# Assuming CE4 is a column in the DataFrame and BV1 is a variable
BV1 = 45  # Assuming the value of $BV$1 is 45

# Define conditions and choices
conditions = [
    me2l_15['Statur TR No FAct'] == "Replanificar +45",
    me2l_15['Statur TR No FAct'] == "Fecha Teórica",
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & ((me2l_15['Categoría LT'] == 0) | (me2l_15['Categoría LT'] == "Dentro de LT")),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 31 a 60 días"),
    (me2l_15['Statur TR No FAct'] == "TR Vencido") & (me2l_15['Categoría LT'] == "De 61 a 90 días")
]

choices = [
    hoy + timedelta(days =45),
    me2l_15['Fecha Facturacion Final'],
    hoy + timedelta(days =30),
    hoy + timedelta(days =60),
    hoy + timedelta(days =90)
]

# Assign the values based on conditions
me2l_15['Fecha No Fact'] = np.select(conditions, choices, default=hoy + timedelta(days =120))


In [123]:
me2l_15['Fecha No Fact'] = pd.to_datetime(me2l_15['Fecha No Fact'])

In [124]:
tabla_final.columns

Index(['AUX', 'Cant. UN. OC', 'Cant. Factura', 'Cantidad en BO'], dtype='object')

In [125]:
me2l_15= me2l_15.merge(tabla_final[['AUX','Cantidad en BO']], left_on='AUX',right_on='AUX', how='left')

In [126]:
me2l_15['Cantidad en BO'] = me2l_15['Cantidad en BO']*me2l_15['Corresponde']

In [127]:
me2l_15['Cantidad en BO'].fillna(0, inplace=True)

In [128]:
me2l_15.columns

Index(['AUX', 'Documento compras', 'Posición', 'Reparto',
       'Cl.documento compras', 'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Fecha de entrega',
       'Hora', 'Fecha entrega estad.', 'Cantidad entrada',
       'Cantidad de salida', 'Cantidad entregada', 'Solicitud de pedido',
       'Pos.solicitud pedido', 'Indicador creación', 'Cantidad de posiciones',
       'Nombre del proveedor', 'Por entregar (cantidad)',
       'Por entregar (valor)', 'Por calcular (cantidad)',
       'Por calcular (valor)', 'NomSector_actual', 'Origen', 'TIPO',
       'Corresponde', 'Fe. ATA', 'Fecha Est. Fact', 'Cant. Factura',
       'Cantidad entrega', 'Qty por Entreg

In [129]:
import numpy as np

# Crear una condición basada en la columna 'TIPO'
condicion = (me2l_15['TIPO'] == 'OEM') & (me2l_15['Por calcular (cantidad)'] == 0)

# Actualizar la columna 'Cantidad en BO' basada en la condición
me2l_15['Cantidad en BO'] = np.where(condicion, 0, me2l_15['Cantidad en BO'])



In [130]:
me2l_15['Por calcular (cantidad)'][me2l_15['AUX']=='420000494760']


Series([], Name: Por calcular (cantidad), dtype: float64)

In [131]:
me2l_15['TIPO'].value_counts()

TIPO
OEM    18120
AXS      220
Name: count, dtype: int64

In [132]:
me2l_15['Cantidad en BO'].sum()

141559.0

In [133]:
me2l_15[me2l_15['AUX'] =='430002269010']

,AUX,Documento compras,Posición,Reparto,Cl.documento compras,Tipo doc.compras,Grupo de compras,Historial pedido/Docu.orden entrega,Fecha documento,Material Antiguo,...,Atraso,Fecha Facturacion Final,Vía (Texto),LT Objetivo,Fe. Confirmación,Fecha OC,Dias LT BO,Categoría LT,Fecha No Fact,Cantidad en BO


In [134]:
me2l_15.columns

Index(['AUX', 'Documento compras', 'Posición', 'Reparto',
       'Cl.documento compras', 'Tipo doc.compras', 'Grupo de compras',
       'Historial pedido/Docu.orden entrega', 'Fecha documento',
       'Material Antiguo', 'Material', 'Texto breve', 'Grupo de artículos',
       'Indicador de borrado', 'Tipo de posición', 'Tipo de imputación',
       'Centro', 'Almacén', 'Cantidad de reparto', 'Cantidad de pedido',
       'Unidad medida pedido', 'Precio neto', 'Moneda', 'Fecha de entrega',
       'Hora', 'Fecha entrega estad.', 'Cantidad entrada',
       'Cantidad de salida', 'Cantidad entregada', 'Solicitud de pedido',
       'Pos.solicitud pedido', 'Indicador creación', 'Cantidad de posiciones',
       'Nombre del proveedor', 'Por entregar (cantidad)',
       'Por entregar (valor)', 'Por calcular (cantidad)',
       'Por calcular (valor)', 'NomSector_actual', 'Origen', 'TIPO',
       'Corresponde', 'Fe. ATA', 'Fecha Est. Fact', 'Cant. Factura',
       'Cantidad entrega', 'Qty por Entreg

In [135]:
me2l_15.to_excel(destino_tubo + "Base Transito para Analisis.xlsx")

In [136]:
side_a = me2l_15[['AUX','Status TR Fact',   'Material','Texto breve','Qty Fact Corr','Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha Facturacion Final','Nombre del proveedor','Grupo de compras']]
side_b = me2l_15[['AUX','Statur TR No FAct','Material','Texto breve','Cantidad en BO',     'Centro','NomSector_actual','Origen','TIPO','Cl.documento compras','Fecha No Fact',          'Nombre del proveedor','Grupo de compras']]

In [137]:
side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
side_b.rename(columns={'Statur TR No FAct':'Status','Cantidad en BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_27184\410670440.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_a.rename(columns={'Status TR Fact':'Status','Qty Fact Corr':'Cantidad','Fecha Facturacion Final':'Fecha'}, inplace=True)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_27184\410670440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_b.rename(columns={'Statur TR No FAct':'Status','Cantidad en BO':'Cantidad','Fecha No Fact':'Fecha'}, inplace=True)


In [138]:
tr_final = pd.concat([side_a, side_b])

In [139]:
tr_final.reset_index()

,index,AUX,Status,Material,Texto breve,Cantidad,Centro,NomSector_actual,Origen,TIPO,Cl.documento compras,Fecha,Nombre del proveedor,Grupo de compras
0,0,430001539910,Facturado Vencido,S203F2602060605,NEUMATICO 22555R19,4.0,201.0,Changan,CL,OEM,ZVOR,2024-09-05 15:19:20.938111,10000717 SAMUEL SALVADOR SUAREZ C,RR4
1,1,430001625610,No Facturado,4311017560R15CPZ,NEUMATICO 17560 R 15 WESLAKE,0.0,201.0,Suzuki,CL,OEM,ZVOR,2024-09-05 15:19:20.938111,10000717 SAMUEL SALVADOR SUAREZ C,RR4
2,2,420000568910,No Facturado,PORTADOCSUZUKI,AXS.PORTA DOCUMENTOS SUZUKI,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-09-05 15:19:20.938111,10001447 I.V. Fabrica de producto,RR2
3,3,420000591710,No Facturado,PORTADCTOKYAL,AXS.PORTA DOCUMENTOS MAZDA,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-09-05 15:19:20.938111,10001447 I.V. Fabrica de producto,RR2
4,4,420000591740,No Facturado,PORTADOCPOER,AXS. PORTA DOCUMENTOS GWM POER,0.0,201.0,Accesorios,CL,AXS,ZSTO,2024-09-05 15:19:20.938111,10001447 I.V. Fabrica de producto,RR2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36675,18335,4300017115360,Fecha Teórica,8210100U1910-F011,FOCO SENALIZADOR ESPEJO IZQ,1.0,201.0,JAC Cars,NaN,OEM,ZVOR,2024-09-19 00:00:00.000000,PU02 DERCO PERU S.A.,RR9
36676,18336,4300017115370,Fecha Teórica,Z-1701114-00-00,RODAMIENTO,1.0,201.0,JAC Cars,NaN,OEM,ZVOR,2024-09-19 00:00:00.000000,PU02 DERCO PERU S.A.,RR9
36677,18337,4300017115380,Fecha Teórica,ZM6T82AF-1701513,SELLO,1.0,201.0,JAC Cars,NaN,OEM,ZVOR,2024-09-19 00:00:00.000000,PU02 DERCO PERU S.A.,RR9
36678,18338,4300017115390,Fecha Teórica,6203400XSZ08A,VIDRIO PUERTA POST DER,1.0,201.0,Great Wall,NaN,OEM,ZVOR,2024-09-19 00:00:00.000000,PU02 DERCO PERU S.A.,RR9


In [140]:
tr_final['Cantidad'].sum()

246899.0

In [141]:
tr_final.to_excel(destino_tubo + "TR FINAL R3.xlsx", index=False)

In [142]:
#MARA
columnas_mara = ['Material_R3','Part_number','Material_dsc','Modelo','Familia', 'Subfamilia', 'Categoría', 'Subcatgería','Sector_dsc']
ruta_mara = ruta_repo.joinpath('MARA_R3.csv')

# Leer el archivo CSV en un DataFrame
df_mara = pd.read_csv(ruta_mara)

print('Ruta Mara: ' + '\n' + str(ruta_mara))

C:\Users\lravlic\AppData\Local\Temp\ipykernel_27184\3838735803.py:6: DtypeWarning: Columns (4,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mara = pd.read_csv(ruta_mara)


Ruta Mara: 
C:\Users\lravlic\Inchcape\Planificación y Compras Chile - Documentos\Bases Indicadores en CSV 2024-08\MARA_R3.csv


In [143]:
import win32com.client
import getpass
usuario = getpass.getuser()

# Initialize SAP GUI Scripting
sap_gui_auto = win32com.client.GetObject("SAPGUI")
application = sap_gui_auto.GetScriptingEngine

# Establish connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the window
session.findById("wnd[0]").maximize()

# Enter transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "me5a"
session.findById("wnd[0]").sendVKey(0)

# Set plant code
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").text = "0201"
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_WERKS-LOW").caretPosition = 4
session.findById("wnd[0]").sendVKey(0)

# Set document type
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_BSART-LOW").caretPosition = 0
session.findById("wnd[0]/usr/btn%_S_BSART_%_APP_%-VALU_PUSH").press()

# Enter multiple values
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = "zsto"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "zspt"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").text = "zvor"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,3]").text = "zipl"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").text = "zatt"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,4]").caretPosition = 4
session.findById("wnd[1]").sendVKey(8)

# Set status
session.findById("wnd[0]/usr/ctxtS_STATU-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtS_STATU-LOW").caretPosition = 0
session.findById("wnd[0]/usr/btn%_S_STATU_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = "a"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "n"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").caretPosition = 1
session.findById("wnd[1]").sendVKey(8)

# Execute
session.findById("wnd[0]").sendVKey(8)

# Export to Excel
session.findById("wnd[0]/tbar[1]/btn[33]").press()
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").setCurrentCell(38, "TEXT")
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").firstVisibleRow = 30
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "38"
session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "ME5A_R3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 8
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the session
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)



In [144]:
me5a_df_r3 = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/ME5A_R3.XLSX")

In [145]:
me5a_df_r3 = me5a_df_r3.merge(cod_actual_df, left_on='Material', right_on='Nro_pieza_fabricante_1',how='left')

In [146]:
me5a_df_r3['Cod_Actual_1'].fillna(me5a_df_r3['Material'], inplace=True)
me5a_df_r3.drop(columns= 'Nro_pieza_fabricante_1', inplace=True)
me5a_df_r3.rename(columns={'Cod_Actual_1':'Ult. Eslabon'}, inplace=True)

In [147]:
me5a_df_r3 = me5a_df_r3[['Solicitud de pedido', 'Clase documento', 'Fecha de solicitud',
       'Pos.solicitud pedido', 'Material','Ult. Eslabon',
       'Texto breve', 'Cantidad solicitada', 'Unidad de medida',
       'Nombre del proveedor', 'Indicador de borrado', 'Status tratamiento',
       'Centro', 'Status tratamiento solicitud pedido', 'Fecha de entrega',
       'Grupo de compras', 'Solicitante', 'Proveedor deseado',
       'Proveedor fijo', 'Reg.info de compras', 'Creado por',
       'Fecha de pedido', 'Nombre del proveedor deseado', 'Pedido',
       'Posición de pedido', 'Proveedor', 'Moneda', 'Precio de valoración',
       'Valor total', 'Petición de oferta',
       'Fecha Petición de oferta', 'Texto bloqueo', 'Cantidad confirmada','Urgencia necesidad'
       ]]

In [148]:
me5a_df_r3.to_excel(destino_tubo + "ME5A R3.xlsx", index=False)

In [149]:
import win32com.client
import getpass
usuario = getpass.getuser()

# Initialize SAP GUI Scripting
sap_gui_auto = win32com.client.GetObject("SAPGUI")
application = sap_gui_auto.GetScriptingEngine

# Establish connection and session
connection = application.Children(0)
session = connection.Children(0)

# Maximize the window
session.findById("wnd[0]").maximize()

# Enter transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "MB52"
session.findById("wnd[0]").sendVKey(0)

# Open value help for plant field and fill in the values
session.findById("wnd[0]/usr/btn%_WERKS_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,1]").text = "0270"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").text = "0501"
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").setFocus()
session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,2]").caretPosition = 4
session.findById("wnd[1]").sendVKey(8)

# Set the variant and execute
session.findById("wnd[0]/usr/ctxtP_VARI").text = "/LRAVLIC"
session.findById("wnd[0]/usr/ctxtP_VARI").setFocus()
session.findById("wnd[0]/usr/ctxtP_VARI").caretPosition = 8
session.findById("wnd[0]").sendVKey(8)

# Export the data
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)"
session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "STOCK_R3.XLSX"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)



In [150]:
stock_r3 = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Bases Transito (python)/STOCK_R3.XLSX")

In [151]:
stock_r3 = stock_r3.merge(cod_actual_df, left_on = 'Material', right_on='Nro_pieza_fabricante_1', how = 'left')

In [152]:
stock_r3.drop(columns='Nro_pieza_fabricante_1', inplace=True)
stock_r3.rename(columns={'Cod_Actual_1':'Ult. Eslabon'}, inplace=True)

In [153]:
stock_r3['Ult. Eslabon'].fillna(stock_r3['Material'], inplace=True)

In [154]:
stock_r3 = stock_r3.merge(df_mara, left_on='Ult. Eslabon', right_on='Material_R3')

In [155]:
stock_r3.to_excel(destino_tubo + "Stock R3.xlsx")